# How to set up a particle code in Peano 4

A very simple demonstration how to use the pidt (particle in dual tree) scheme
within Peano 4. We create a bunch of particles. They are randomly distributed within the unit square. After that, we let Peano 4's particle toolbox puzzle out which particle belongs to which vertex logically, and if we have to refine adaptively if we want to have at most 4 particles in a cell. We eventually terminate. So no particle moves actually.

Create a project and configure it to end up in a subnamespace (and thus subdirectory). 

In [ ]:
import os

import peano4
import peano4.dastgen2
import peano4.toolbox
import peano4.toolbox.particles
import dastgen2


In [ ]:
project = peano4.Project( ["examples", "particles"], "particle-setup", "." )

## Prepare code

Peano's API does not know which settings to use on the present system. To make it copy/clone the settings identified by ./configure, we ask it to parse the generated configuration scripts. 


In [ ]:
build_mode = peano4.output.CompileMode.Asserts
project.output.makefile.parse_configure_script_outcome( "../../.." )
project.output.makefile.set_mode( build_mode )

We now know all about your Peano installation in the Python script, i.e. the project knows where all the libraries are (implicitly, as you have handed in the path) and it knows which options to use to build further code on top of your library. If you have used certain environment variables or modules when you have built the Peano libraries, it is important that the same environment has been used before you start up the Jupyter notebook.

## Model the particle

We create the particle through the toolbox. A particle by definition has a position and a cut-off radius which is all we need in this example. However, you can, in principle, add an arbitrary number of attributes to each particle. You can also have different particle types if you want. We discuss this in the next example when particles actually move.

In [ ]:
particle  = peano4.toolbox.particles.Particle( "Particle" )
particles = peano4.toolbox.particles.ParticleSet( particle )

This particle has to be added to the project. This way the project knows that we are working with a particular particle. Next, we associate the particle with the vertices, as we work with the pidt (particle in dual tree) algorithm. In principle, we can also associate particles with cells or faces, but then we do not have Peano's toolbox support to query which particles are around (aka in the area around a cell) or to move particles between different processes.

In [ ]:
project.datamodel.add_global_object(particle)
project.datamodel.add_vertex(particles)

Once we have completed this first setup, you will recognise that the above code snippet generates a simple C++ class with a number of additional attributes. It also creates some MPI routines. For your own project later on, you can either rely on this workflow or you write your own particle classes. In the first variant, the actual C++ particle implementations will always be generated. So the C++ class is a dump particle storage thing subject to some aspects such as MPI data transfer. In return, you can model additional fields et al all through Python. If you decide later to write your own particle C++ classes, you have all the freedom to add routines to the particle, additional fields, and so forth. You work with the whole power of C++ and its encapsulation. In return, you will have to ensure that the particle fits to some basic signature so Peano's toolbox finds all the functions it expects to have. 

## Model the algorithmic phases

Our introductory example orbits around a where nothing happens. Which is not true. This first setup creates a regular grid, inserts some particles, and plots the outcome. Peano will later on intermix and parallelise various activities behind the scene, but as a user we can always think in algorithmic steps on the Python API level which run one after another. We will therefore need three algorithmic steps: CreateGrid, Init and Plot.

### Create an initial mesh

Creating a step always consists of three activities: create the algorithmic step with a unique name, add the actual action sets, i.e. what is done in this algorithmic step, and finally add all of it to the project. 

In [ ]:
mesh_size = 0.2

create_grid = peano4.solversteps.Step( name="CreateGrid", add_user_defined_actions=False )
create_grid.add_action_set( peano4.toolbox.CreateRegularGrid(mesh_size) )
project.solversteps.add_step(create_grid)

The first line above defines a "solver step" CreateGrid. It also tells Peano that we will not add anything to this step manually. When we skip this flag (default is True), we'll get some C++ code with 

      // @todo insert your stuff here

which we could simply ignore, but passing in False right from the start works, too, and it reduces the number of overall files. In the second line, we add a pre-defined action set. We could actually implement something ourselves which creates a regular grid, but Peano offers something like that already. So why bother. Finally, we add the solver step to our project.

You might ask why we have to use the particles and the marker. We don't. But we want to ensure that all data are properly initialised when we create the grid, so we add them here. In general, it is a good idea to make all steps use the same data - even if they don't need it. You can tune such behaviour later.

### Init setup (add the particles)

Once we have created our grid, we will want to insert the particles. This is our next solver step. Different to the creation, we use some further actions this time: 

- We make it use the particles.
- We rely on some further pre-defined particles which ensure that any particle that we insert is assigned to correct vertex.
- We tell Peano that we would like to inject some user code so we can insert particles into unrefined cells.

For all of these steps, Peano's particle toolbox provides pre-defined action sets. One of the pre-defined action sets relies on a marker per cell to store information about the tree. So before we start, we create this marker, add it ot the data model and then use the tree analysis over this marker.

In [ ]:
particle_tree_analysis = peano4.toolbox.particles.ParticleTreeAnalysis(particles)
project.datamodel.add_cell(particle_tree_analysis.cell_marker)   # read docu of ParticleTreeAnalysis

initialisation_snippet = """
 // we could do something intelligent here, but we actually
 // just set the cut-off radius
 particle->setCutOffRadius(0.001);
"""

init_setup = peano4.solversteps.Step( name="Init", add_user_defined_actions=False )
init_setup.use_vertex(particles)
init_setup.use_cell(particle_tree_analysis.cell_marker)
init_setup.add_action_set( peano4.toolbox.particles.UpdateParticleGridAssociation(particles) )
init_setup.add_action_set( peano4.toolbox.particles.InsertRandomParticlesIntoUnrefinedCells(
  particle_set=particles,
  average_distance_between_particles=mesh_size/10,
  initialisation_call=initialisation_snippet,
  noise=True ))
init_setup.add_action_set( peano4.toolbox.particles.ParticleAMR(particles,particle_tree_analysis) )
init_setup.add_action_set( particle_tree_analysis )
project.solversteps.add_step(init_setup)

The above snippet supplements the step with some AMR routines. That makes sense: After we've inserted particles, we might want to refine. As I wrote *after* we have to insert this action set after the insertion.

There's a caveat with the code above: It uses the particles sets and the markers of particle_tree_analysis. However these guys are not properly initialised when we hit the initialisation. We also have to add them to the grid creation so they are properly in place when we get here:

In [ ]:
create_grid.use_vertex(particles)
create_grid.use_cell(particle_tree_analysis.cell_marker)

### Plot outcome 

Finally, we will have to plot the outcome (or any snapshot). In this setup, I want each of my dumps to produce two types of files: particle data and mesh data. For the mesh, I rely on the toolbox's grid plotting.

In [ ]:
print_solution = peano4.solversteps.Step( "Plot", add_user_defined_actions=False )
print_solution.use_vertex(particles)
print_solution.use_cell(particle_tree_analysis.cell_marker)
print_solution.add_action_set( peano4.toolbox.PlotGridInPeanoBlockFormat( filename="grid", time_stamp_evaluation=peano4.toolbox.PlotGridInPeanoBlockFormat.CountTimeSteps ) )
print_solution.add_action_set( peano4.toolbox.particles.PlotParticlesInVTKFormat( "particles", particles, time_stamp_evaluation=peano4.toolbox.particles.PlotParticlesInVTKFormat.CountTimeSteps ) )
project.solversteps.add_step(print_solution)


## Generate the actual C++ code

Next, we generate the actual Peano 4 C++ code:

In [ ]:
project.generate()

we run through the standard triad of instructions of any Peano Python code. You can skip the first two steps if you want as the script then will automatically invoke the previous steps. The other way round, it is always admissible to only generate stuff, e.g., but to build and run the project through a command line.

## Write the main

There is one file that the Python API cannot write for you: That's the main file. In the present case, the main file is close to trivial: 


    int gridConstructionSteps = 0;
    while (peano4::parallel::SpacetreeSet::getInstance().getGridStatistics().getStationarySweeps()<5) {
      logInfo( "main()", "run a grid construction step and insert particles" );
      examples::particles::observers::CreateGrid createObserver;
      peano4::parallel::SpacetreeSet::getInstance().traverse(createObserver);
      gridConstructionSteps++;
    }

    logInfo( "main()", "finished grid construction after " << gridConstructionSteps << " steps, start timestepping" )

    examples::particles::observers::Init initObserver;
    peano4::parallel::SpacetreeSet::getInstance().traverse(initObserver);
    logInfo( "main()", "added particles" )

    examples::particles::observers::Plot plotObserver;
    peano4::parallel::SpacetreeSet::getInstance().traverse(plotObserver);
    logInfo( "main()", "dumped initial condition" )

We don't have to write the surrounding code ourself: If we run generate() as done above, we'll get a main file which we can use as a starting point. Once we have modified this blueprint, subsequent generate() calls won't overwrite our main files anymore. So we are on the save side.


## Build the code

With all in place, we can invoke the build. This could be a plain command line call to make (under the hood, Peano's Python API generates makefiles), but we can also use the Python API.

In [ ]:
project.build()

## Run code

Before we run any code, we clean up, i.e. remove all old output files.

In [ ]:
output_files = [ f for f in os.listdir(".") if f.endswith(".peano-patch-file") or f.endswith(".vtu") or f.endswith(".pvd") ]
for f in output_files:
  os.remove(f)


I could now run the code via a Python command

    #success = project.run( args=[], prefix=["mpirun", "-n", "1"] )
    
but instead I here decide to access the terminal directly via the notebook. Before you invoke the run, some remarks:

- Peano can build code in different versions. By default, I use the assert mode here which is excellent to find errors in the code. For production runs, you should use the release mode (or tracing if you wanna profile).
- Our code is not yet MPI-ready. Shared memory would work out of the box, but is not active as we have not yet provided a load balancing criterion. MPI requires slightly more work. So keep you problem smallish.
- If you translate in release mode, you'll get a lot of output. Tweak the filters in the main (or use a filter file) to study only those output you are actually interested in.

In [ ]:
# success = project.run( args=["--threads", "1"], prefix=["mpirun", "-n", "1"] )
!./peano4

## Visualisation

Peano 4 and its toolboxes provide multiple scripts to postprocess and visualise all output. You can directly visualise runtimes or load distributions with matplotlib, e.g. We don't go down this route in this simple example. We focus only on the particle and grid data.

The particle data has been written in a mainstream formats (vtk) that we can load directly into Paraview. This is fine for the time being. For complex simulations, you will want to use tailored, Peano 4-specific formats instead.

In [ ]:
from IPython.display import Image
!ls ../../python/peano4/toolbox/particles
Image("particles.png")

The grid is written in Peano's block format which relatively efficient. However, this format cannot be processed by Paraview directly. You first have to convert it. The guidebook discusses various ways to do this, but the Paraview's Python command is perhaps the simplest way:

In [ ]:
!pvpython ../../../python/peano4/visualisation/render.py --filter-fine-grid grid.peano-patch-file

We can now combine the particle visualisation with the actual grid. We see exactly what we would expect: A regular grid with roughly the same number of particles (10x10) per cell.

In [ ]:
from IPython.display import Image
!ls ../../python/peano4/toolbox/particles
Image("particles-with-grid.png")